In [23]:
import os
os.chdir("../")

from collections import Counter
from tqdm.notebook import tqdm
from semiolog import util
from functools import reduce
import operator
from importlib import reload
from datetime import datetime

import temp
import semiolog as slg

In [2]:
semiotic = slg.Cenematic("fr_wiki")

Vocabulary will not be loaded from file.



In [28]:
corpus = semiotic.corpus.train

normalizer = slg.syntagmatic.tokenizer.normalizers.Sequence(semiotic.vocab.config.normalizer)

chain_list = []

alphabet = Counter()

for sent in tqdm(corpus, desc="Chain List & Alphabet:"):
    sent = normalizer.normalize(sent)
    sent = list(sent)
    if sent !=[]:
        chain_list += sent
        alphabet.update(Counter(sent))

In [29]:
reload(temp)

<module 'temp' from '/Users/Gianni/semiolog/wip/temp.py'>

In [30]:
for i in range(5):
    now = datetime.now()
    result = util.multiprocessing(temp.find_best_pair, temp.parallel_chain(chain_list,4))

    parallel = reduce(operator.add, result)
        
    parallel = parallel.most_common()
    print(datetime.now() - now)

0:00:02.837991
0:00:02.838272
0:00:03.141748
0:00:03.106652
0:00:02.933724


In [25]:
for i in range(5):
    now = datetime.now()
    
    sequential = temp.find_best_pair(chain_list)
    sequential = sequential.most_common()
    
    print(datetime.now() - now)

0:00:05.527317
0:00:05.145561
0:00:05.272142
0:00:05.325645
0:00:05.272945


In [19]:
 parallel == sequential

True

In [33]:
semiotic.vocab.build(vocab_size = 1627)


Vocabulary built


In [ ]:
semiotic.vocab.build(vocab_size = 1627, parallel=True, parallel_mode="process")